In [3]:
import numpy as np
import pandas as pd
import pickle

In [4]:
def handle_categorical_columns(df):
    """
    The columns "srcAS", "destAS", "srcCC", "destCC" contain only few distinct values.
    It is more efficient to treat them as categorical values and replace each category 
    with an unique integer.
    """
    categorical_columns = ["srcAS", "destAS", "srcCC", "destCC"]
    for column_name in categorical_columns:
        print(f"Generating column '{column_name}_cat'")
        df[f"{column_name}_cat"] = df[column_name].astype("category").cat.codes
        if df[f"{column_name}_cat"].min() < 0:
            df[f"{column_name}_cat"] = df[f"{column_name}_cat"] + abs(df[f"{column_name}_cat"].min())
        df[f"{column_name}_cat"] = df[f"{column_name}_cat"].astype("category")
        df = df.drop([column_name], axis=1)
    return df

def generate_columns_for_sequences(df):
    """
    The columns "lat", "long", "IPSource", "geoCC" and "ASN" contain lists.
    This function creates a column for each item in thos lists. As there are at maximum 39
    values in those lists we need to create 39 additional columns for each of the columns named before.
    If a list is shorter than the maximum value the corresponding columns are filled with default values.
    """
    max_hops = df["num_hops"].max()
    sequence_columns = [("lat", 0.0), ("long", 0.0), ("IPsource", 0), ("geoCC", float("nan")), ("ASN", 0)]
    for column_name, default_value in sequence_columns:
        print(f"Generating new columns for '{column_name}'")
        for i in range(max_hops):
            df[f"{column_name}_{i}"] = default_value
            df[f"{column_name}_{i}"] = df[column_name].apply(lambda x: x[i] if len(x) >= (i+1) else default_value)
            if type(default_value) == int:
                df[f"{column_name}_{i}"] = df[f"{column_name}_{i}"].astype("category")
            # Normalize 'lat' and 'long' columns
            if column_name == "lat":
                df[f"{column_name}_{i}"] = df[f"{column_name}_{i}"] / 90
            if column_name == "long":
                df[f"{column_name}_{i}"] = df[f"{column_name}_{i}"] / 180
        df = df.drop([column_name], axis=1)
    return df

def replace_geoCC_codes_with_integers(df, df_countries):
    """
    The column "geoCC" contains a list of country codes. These should be mapped to integeres.
    """
    max_hops = df["num_hops"].max()
    # Iterate over all geoCC columns
    for i in range(max_hops):
        column_name = f"geoCC_{i}"
        print(f"Replacing strings with category codes in {column_name}")
        for af, af_code in zip(df_countries.index, df_countries["A2Code_cat"]):
            mask = df[column_name] == af
            df.loc[mask, column_name] = af_code + 1 # we have to add 1, because 0 is reserved for nan
        # Replace nan columns with 0
        mask = (df[column_name].isna()) | (df[column_name] == 'nan')
        df.loc[mask, column_name] = 0
        df[column_name] = df[column_name].astype(int)
        df[column_name] = df[column_name].astype("category")
    return df    

In [5]:
# Source: https://www.iban.com/country-codes
df_countries = pd.read_csv("country_codes.tsv", sep='\t')
df_countries["A2Code_cat"] = df_countries["A2Code"].astype("category").cat.codes
df_countries = df_countries.set_index("A2Code")
df_countries

,Country,A3Code,Numeric,A2Code_cat
A2Code,,,,
AF,Afghanistan,AFG,4,2
AL,Albania,ALB,8,5
DZ,Algeria,DZA,12,61
AS,American Samoa,ASM,16,10
AD,Andorra,AND,20,0
...,...,...,...,...
XK,XK,XXX,0,244
YE,Yemen,YEM,887,246
ZM,Zambia,ZMB,894,249


In [6]:
subset = "training"
df = pd.read_pickle(f"dataset_b_noisy_7_agents_{subset}.pkl")

In [7]:
df

,srcAS,destAS,srcCC,destCC,num_hops,lat,long,IPsource,geoCC,ASN,vf,combined,geo,owner
trid,,,,,,,,,,,,,,
1328731,14061,42999,US,TR,10,"[50.115520000000004, 50.115520000000004, 50.11...","[8.68417, 8.68417, 8.68417, 8.67185, 9.491, 8....","[1, 2, 1, 1, 3, 1, 3, 3, 2, 3]","[DE, DE, DE, DE, DE, DE, TR, TR, TR, TR]","[14061, 14061, 2914, 2914, 2914, 9121, 9121, 9...",0,0,0,0
5183622,138062,3605,ID,GU,1,[22.285521],[114.1577],[1],[HK],[24482],0,0,0,0
3258755,14061,24173,US,VN,10,"[40.714271999999994, 40.714271999999994, 40.71...","[-74.005966, -74.005966, -74.005997, -122.3320...","[1, 1, 1, 1, 1, 1, 1, 3, 3, 3]","[US, US, US, US, JP, HK, HK, HK, VN, VN]","[14061, 14061, 2914, 2914, 2914, 2914, 2914, 2...",0,0,0,0
2660757,16276,7171,FR,US,13,"[0.0, 0.0, 0.0, 0.0, 1.28967, 43.296669, 48.86...","[0.0, 0.0, 0.0, 0.0, 103.850098, 5.37639, 2.34...","[4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[nan, nan, nan, nan, SG, FR, FR, FR, FR, US, U...","[0, 0, 0, 0, 16276, 16276, 16276, 6461, 6461, ...",0,0,0,0
1730541,20473,12479,US,ES,7,"[48.860001000000004, 48.860001000000004, 48.86...","[2.34444, 2.34444, 2.34444, 5.381069999999999,...","[1, 1, 1, 2, 1, 1, 1]","[FR, FR, FR, FR, ES, ES, ES]","[3257, 3257, 3257, 5511, 5511, 5511, 5511]",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491862,14061,3549,US,US,7,"[50.115520000000004, 50.115520000000004, 50.11...","[8.68417, 8.68417, 8.67185, 8.67185, 8.67185, ...","[2, 1, 1, 1, 1, 1, 3]","[DE, DE, DE, DE, DE, GB, GB]","[14061, 14061, 2914, 2914, 3356, 3549, 3549]",0,0,0,0
1616685,20473,27385,US,US,25,"[48.860001000000004, 48.860001000000004, 50.11...","[2.34444, 2.34444, 8.67185, 8.67185, 8.68417, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, ...","[FR, FR, DE, DE, DE, DE, GB, GB, GB, GB, US, U...","[1299, 1299, 1299, 1299, 209, 3561, 3561, 3561...",0,0,0,0
1672632,20473,6830,US,AT,6,"[48.860001000000004, 48.860001000000004, 48.20...","[2.34444, 2.34444, 16.37208, 26.10626, 26.1062...","[1, 1, 1, 1, 1, 1]","[FR, FR, AT, RO, RO, RO]","[1299, 1299, 1299, 1299, 1299, 6830]",0,0,0,0


In [8]:
# Prepare data for machine learning, that means transform all columns to numerical values
df = handle_categorical_columns(df)
df = generate_columns_for_sequences(df)
df = replace_geoCC_codes_with_integers(df, df_countries)

Generating column 'srcAS_cat'
Generating column 'destAS_cat'
Generating column 'srcCC_cat'
Generating column 'destCC_cat'
Generating new columns for 'lat'
Generating new columns for 'long'
Generating new columns for 'IPsource'
Generating new columns for 'geoCC'


/tmp/ipykernel_26768/1075805575.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
/tmp/ipykernel_26768/1075805575.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
/tmp/ipykernel_26768/1075805575.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

Generating new columns for 'ASN'


/tmp/ipykernel_26768/1075805575.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
/tmp/ipykernel_26768/1075805575.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
/tmp/ipykernel_26768/1075805575.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

Replacing strings with category codes in geoCC_0
Replacing strings with category codes in geoCC_1
Replacing strings with category codes in geoCC_2
Replacing strings with category codes in geoCC_3
Replacing strings with category codes in geoCC_4
Replacing strings with category codes in geoCC_5
Replacing strings with category codes in geoCC_6
Replacing strings with category codes in geoCC_7
Replacing strings with category codes in geoCC_8
Replacing strings with category codes in geoCC_9
Replacing strings with category codes in geoCC_10
Replacing strings with category codes in geoCC_11
Replacing strings with category codes in geoCC_12
Replacing strings with category codes in geoCC_13
Replacing strings with category codes in geoCC_14
Replacing strings with category codes in geoCC_15
Replacing strings with category codes in geoCC_16
Replacing strings with category codes in geoCC_17
Replacing strings with category codes in geoCC_18
Replacing strings with category codes in geoCC_19
Replacing 

In [9]:
df

,num_hops,vf,combined,geo,owner,srcAS_cat,destAS_cat,srcCC_cat,destCC_cat,lat_0,...,ASN_29,ASN_30,ASN_31,ASN_32,ASN_33,ASN_34,ASN_35,ASN_36,ASN_37,ASN_38
trid,,,,,,,,,,,,,,,,,,,,,
1328731,10,0,0,0,0,1,23574,3,208,0.556839,...,0,0,0,0,0,0,0,0,0,0
5183622,1,0,0,0,0,4,1233,1,87,0.247617,...,0,0,0,0,0,0,0,0,0,0
3258755,10,0,0,0,0,1,12293,3,223,0.452381,...,0,0,0,0,0,0,0,0,0,0
2660757,13,0,0,0,0,2,2398,0,215,0.000000,...,0,0,0,0,0,0,0,0,0,0
1730541,7,0,0,0,0,3,5254,3,64,0.542889,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491862,7,0,0,0,0,1,1205,3,215,0.556839,...,0,0,0,0,0,0,0,0,0,0
1616685,25,0,0,0,0,3,14187,3,215,0.542889,...,0,0,0,0,0,0,0,0,0,0
1672632,6,0,0,0,0,3,2255,3,12,0.542889,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.to_pickle(f"dataset_{subset}.pkl.gz", compression='gzip')